In [1]:
!pip install tqdm

Deze notebook is bedoelt om een rasterdataset op te knippen met een feature layer. Hierbij wordt ook een deel van de raster boven de waarde "n" omgezet naar "NoData". Maak van tevoren een map aan in je workspace genaamd DEM_CLIP

In [11]:
import arcpy, os
import numpy as np
from tqdm import tqdm
from osgeo import gdal
arcpy.env.addOutputsToMap = False   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True


# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

    
#De Shapefile die je wilt gebruiken voor het opknippen van de raster (BR_Peilgebieden, BR_Afvoergebieden, ect.)
clipShapefile  = "West_test"

#De raster die je wilt gebruiken als input (AHN3, AHN4, ect.)
#Hou rekening met het Talut van de watergangen en neem indien nodig stappen om deze uit de DEM te filteren
rasterlist = ["AHN3_HDSR_FILL_ISV_NO.TIF"]
raster_input = "AHN3_HDSR_FILL_ISV_NO.TIF"

In [12]:
Percentile_value_min = 15            # Het Percentiel [0-100] waaronder je alle data wilt omzetten naar nodata [Default = 0]
Percentile_value_max = 20              # Het Percentiel [0-100] waarboven je alle data wilt omzetten naar nodata [Default = 10]
Raster_Clip          = "Test26042023_1"   # De folder waar de geclipte raster worden opgeslagen

arcpy.management.CreateFolder(directory, Raster_Clip)
outputWorkspace = directory 
arcpy.env.workspace = outputWorkspace
outputWorkspace_clip = os.path.join(outputWorkspace, Raster_Clip)
 

Het onderstaande codeblok bevat de functies die later worden aangeroepen in het script, het gaat hierbij om twee functies:

- clip_and_set_null

Deze functie clipt de rasterkaart op naar de shapefile die is aangegeven met de variabele "clipShapefile", deze worden opgeslagen in de map met naam "Raster_Clip" (in te vullen bij variabele hierboven). Daarna gebruikt de functie de waarden van "Percentile_value_min" en "Percentile_value_max" om een percentiel bereik van de data om te zetten in NoData, de resulterende kaarten worden opgeslagen in de map met aanduiding "STAT_...".



- set_null

Deze functie gaat verder waar de volgende functie stopt. Als de rasterkaart eenmaal is opgeknipt is het niet nodig dit nog een keer te doen voor een nieuw percentiel bereik. De resulterende kaarten worden opgeslagen in de map met aanduiding "STAT_...".

Het is niet de bedoeling om deze functies aan te passen! 

In [22]:
# NIET AANKOMEN
def clip_and_set_null(input_raster, feature_shapefile, output_folder, n1, n2):
    
    if n1 >= n2:
        raise ValueError('"Percentile_value_min" cannot be higher or equal to "Percentile_value_max"')
        
    arcpy.management.CreateFolder(output_folder, "STAT_{}_{}".format(n1, n2))
    output_folder_null = os.path.join(output_folder, "STAT_{}_{}".format(n1, n2))
    
    # Create a search cursor to iterate through the features in the shapefile
    with arcpy.da.SearchCursor(feature_shapefile, ['OID@', 'CODE']) as cursor:
        for row in tqdm(cursor):
            # Get the OID and name values for the current feature
            feature_oid = row[0]
            feature_name = row[1]
            
            # Create a feature layer with only the current feature
            arcpy.MakeFeatureLayer_management(feature_shapefile, 'clip_layer', ' "FID" = {}'.format(feature_oid))

            # Set the output path for the clipped raster based on the feature name
            output_raster = os.path.join(output_folder, '{}.tif'.format(feature_name))

            # Clip the input raster with the feature layer
            arcpy.Clip_management(input_raster, '#', output_raster, 'clip_layer', '99', 'ClippingGeometry')
            
            # Calculate the percentile values for n1 and n2
            try:
                ds = gdal.Open(output_raster)
                band = ds.GetRasterBand(1)
                arr = np.array(band.ReadAsArray(), dtype="f4")
                arr[arr==99] = np.nan
                ds = None
            except MemoryError:
                print(raster + " is too big")
                continue
            except:
                print(raster + " something else went wrong")
                continue
            
            perc_n1 = float(numpy.nanpercentile(arr, n1))
            perc_n2 = float(numpy.nanpercentile(arr, n2))
            
            # Set the values outside the percentile range to null
            null_raster = arcpy.sa.SetNull(output_raster, output_raster, 'VALUE < {} OR VALUE > {}'.format(perc_n1, perc_n2))

            # Save the null raster to a new file
            null_raster.save(os.path.join(output_folder_null, '{}_null.tif'.format(feature_name)))


            # Delete the clip layer and intermediate data
            arcpy.Delete_management('clip_layer')
            arcpy.ClearWorkspaceCache_management() 
            
    # Set the workspace environment
    arcpy.env.workspace = output_folder_null

    # Create a list of all the raster files in the workspace
    raster_list_null = arcpy.ListRasters("*", "TIF")

    # Set the output mosaic raster name 
    output_raster = "_mosaic_{}_{}.tif".format(n1, n2)

    # Use the MosaicToNewRaster_management function to mosaic the input rasters
    arcpy.MosaicToNewRaster_management(raster_list_null, output_folder_null, output_raster, pixel_type="32_BIT_FLOAT", mosaic_method="LAST", cellsize="0,5", number_of_bands="1", mosaic_colormap_mode="FIRST")


    return  perc_n1, perc_n2

def set_null(input_raster_folder, n1, n2):
    
    if n1 >= n2:
        raise ValueError('"Percentile_value_min" cannot be higher or equal to "Percentile_value_max"')
    
    arcpy.env.workspace = input_raster_folder
    rasterlist = arcpy.ListRasters('*.tif*') # Get a list of input rasters
    outputWorkspace = os.path.join(input_raster_folder, "STAT_{}_{}".format(n1, n2))
    arcpy.management.CreateFolder(input_raster_folder, "STAT_{}_{}".format(n1, n2))
    
    for raster in tqdm(rasterlist):
        # Calculate the percentile values for n1 and n2
        
        try:
            Raster_tmp = arcpy.sa.Raster(raster)
            ds = gdal.Open(Raster_tmp.catalogPath)
            band = ds.GetRasterBand(1)
            arr = np.array(band.ReadAsArray(), dtype="f4")
            arr[arr==99] = np.nan
            perc_n1 = float(numpy.nanpercentile(arr, n1))
            perc_n2 = float(numpy.nanpercentile(arr, n2))
        except MemoryError:
            print(raster + " is too big")
            continue
        except:
            print(raster + " something else went wrong")
            continue

        # Set the values outside the percentile range to null
        null_raster = arcpy.sa.SetNull(raster, raster, 'VALUE < {} OR VALUE > {}'.format(perc_n1, perc_n2))

        # Save the null raster to a new file
        raster_string = raster.replace(".tif", "")
        null_raster.save(os.path.join(outputWorkspace, '{}_null.tif'.format(raster_string)))
        arcpy.ClearWorkspaceCache_management()
        
    # Set the workspace environment
    arcpy.env.workspace = outputWorkspace

    # Create a list of all the raster files in the workspace
    raster_list_null = arcpy.ListRasters("*", "TIF")

    # Set the output mosaic raster name 
    output_raster = "_mosaic_{}_{}.tif".format(n1, n2)

    # Use the MosaicToNewRaster_management function to mosaic the input rasters
    arcpy.MosaicToNewRaster_management(raster_list_null, outputWorkspace, output_raster, pixel_type="32_BIT_FLOAT", mosaic_method="LAST", cellsize="0,5", number_of_bands="1", mosaic_colormap_mode="FIRST")

In [16]:
clip_and_set_null(raster_input, clipShapefile, outputWorkspace_clip, Percentile_value_min, Percentile_value_max)

12it [16:55, 84.61s/it]


(-2.0209999084472656, -1.9989999532699585)

In het onderstaande code blok kan je ook de kaarten maken voor een ander percentiel bereik. Pas de waarden aan en run de cel, in de huidige map zal, naast de al bestaande folder van de vorrige cel, nog een folder worden aangemaakt waar de nieuwe kaarten in komen te staan. 

In [23]:
Percentile_value_min = 20               # Het Percentiel [0-100] waaronder je alle data wilt omzetten naar nodata [Default = 0]
Percentile_value_max = 30            # Het Percentiel [0-100] waarboven je alle data wilt omzetten naar nodata [Default = 10]

set_null(outputWorkspace_clip, Percentile_value_min, Percentile_value_max)

100%|##########| 12/12 [11:26<00:00, 57.17s/it]
